<a href="https://colab.research.google.com/github/FatemehRanjbar/Image_process/blob/main/Multi_Page_TIFF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdf2image

In [ ]:
pip
install
Pillow

#Resources
https://joblib.readthedocs.io/en/latest/auto_examples/compressors_comparison.html

https://linkedin.com/pulse/python-image-compression-algorithm-femi-temitope-bodunde

https://pywavelets.readthedocs.io/en/latest/

https://medium.com/@mhconradt/image-compression-from-scratch-in-python-ee357fea3f9e

#Compression methods used:
#LZMA
#jpeg
#tiff_deflate
#tiff_jpeg
#tiff_lzw
#tiff_adobe_deflate
#PyWavelets

#First


In [ ]:
from PIL import Image
import os


# Interface for Image Compression
class ImageCompressor:
    def compress_image(self, image, compression_level):
        raise NotImplementedError("Subclasses should implement this!")


# JPEG/PNG to TIFF Converter implementing the Image Compressor Interface
class JpegToTiffConverter(ImageCompressor):
    def compress_image(self, image, compression_level):
        return image.convert("RGB").save(compression_level)

    def convert_images_to_tiff(self, image_list, output_file, compression="jpeg", quality=70):
        # Save images as multi-page TIFF with compression
        if image_list:
            image_list[0].save(output_file, save_all=True, append_images=image_list[1:], compression=compression,
                               quality=quality)
            print(f"Images successfully saved to {output_file}")


# Interface Segregation for different operations
class ImageLoader:
    def load_images(self, folder_path):
        raise NotImplementedError("Subclasses should implement this!")


class JpegImageLoader(ImageLoader):
    def load_images(self, folder_path):
        image_list = []
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(folder_path, filename)
                with Image.open(file_path) as img:
                    # Resize the image to reduce file size
                    img = img.resize((int(img.width * 0.70), int(img.height * 0.70)))  # Resize to 75% of the original
                    compressed_img = img.convert("RGB")
                    image_list.append(compressed_img)
        return image_list


# Entry point with Dependency Injection
class ImageProcessor:
    def __init__(self, converter: ImageCompressor, loader: ImageLoader):
        self.converter = converter
        self.loader = loader

    def process_images(self, folder_path, output_file, compression, quality):
        image_list = self.loader.load_images(folder_path)
        self.converter.convert_images_to_tiff(image_list, output_file, compression, quality)


def main():
    folder_path = "Images"

    output_file = "output_compressed.tif"

    compression = "jpeg"  # Using lossy JPEG compression within the TIFF
    quality = 70  # Adjust quality; lower values ​​= higher compression

    # Dependency Injection for flexibility
    converter = JpegToTiffConverter()
    loader = JpegImageLoader()
    processor = ImageProcessor(converter, loader)

    processor.process_images(folder_path, output_file, compression, quality)
    print(f"Conversion completed. Output saved as {output_file}.")


if __name__ == "__main__":
    main()


#Secound

In [ ]:
from PIL import Image, TiffImagePlugin
import os


# Interface for Image Compression
class ImageCompressor:
    def compress_image(self, image, compression_params):
        raise NotImplementedError("Subclasses should implement this!")


# JPEG/PNG to TIFF Converter implementing the Image Compressor Interface
class JpegToTiffConverter(ImageCompressor):
    def compress_image(self, image, compression_params):
        compressed_image = image.convert("RGB")
        compressed_image.save(compression_params['output_path'], format='TIFF',
                              compression=compression_params['compression'], quality=compression_params['quality'])
        return compressed_image

    def convert_images_to_tiff(self, image_list, output_file, compression_params):
        # Save images as multi-page TIFF with compression
        if image_list:
            image_list[0].save(
                output_file,
                save_all=True,
                append_images=image_list[1:],
                compression=compression_params['compression'],
                resolution=compression_params['resolution'],
                quality=compression_params['quality']
            )
            print(f"Images successfully saved to {output_file}")


# Interface Segregation for different operations
class ImageLoader:
    def load_images(self, folder_path):
        raise NotImplementedError("Subclasses should implement this!")


class JpegImageLoader(ImageLoader):
    def load_images(self, folder_path):
        image_list = []
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(folder_path, filename)
                with Image.open(file_path) as img:
                    # Resize the image to reduce file size
                    new_width = int(img.width * 0.50)
                    new_height = int(img.height * 0.50)
                    img = img.resize((new_width, new_height), Image.ANTIALIAS)
                    image_list.append(img.convert("RGB"))
        return image_list


# Entry point with Dependency Injection
class ImageProcessor:
    def __init__(self, converter: ImageCompressor, loader: ImageLoader):
        self.converter = converter
        self.loader = loader

    def process_images(self, folder_path, output_file, compression_params):
        image_list = self.loader.load_images(folder_path)
        self.converter.convert_images_to_tiff(image_list, output_file, compression_params)


def main():
    folder_path = "Images"

    output_file = "output_compressed.tif"

    compression_params = {
        'compression': 'jpeg',  # Using lossy JPEG compression within the TIFF
        'quality': 75,  # Adjust quality; lower values = higher compression
        'output_path': output_file,
        'resolution': (195, 195)  # Setting resolution can also impact file size
    }

    # Dependency Injection for flexibility
    converter = JpegToTiffConverter()
    loader = JpegImageLoader()
    processor = ImageProcessor(converter, loader)

    processor.process_images(folder_path, output_file, compression_params)
    print(f"Conversion completed. Output saved as {output_file}.")


if __name__ == "__main__":
    main()


#third

In [ ]:
import os
from PIL import Image


class ImageProcessor:
    def __init__(self, input_files, output_file, target_size):
        self.input_files = input_files
        self.output_file = output_file
        self.target_size = target_size  # in MB

    def read_images(self):
        images = []
        for file in self.input_files:
            img = Image.open(file)
            images.append(img)
        return images

    def save_as_multipage_tiff(self, images, quality=95):
        # Save images list as a multipage TIFF
        images[0].save(self.output_file, save_all=True, append_images=images[1:], compression='jpeg', quality=quality)

    def compress_tiff(self, images):
        # Make sure to iteratively compress until target size is met
        best_quality = 95
        final_size = os.path.getsize(self.output_file) / 1024 ** 2
        while final_size > self.target_size and best_quality > 10:
            best_quality -= 5
            self.save_as_multipage_tiff(images, quality=best_quality)
            final_size = os.path.getsize(self.output_file) / 1024 ** 2
        return final_size

    def process(self):
        images = self.read_images()
        self.save_as_multipage_tiff(images)
        final_size = self.compress_tiff(images)
        print(f"Final output file size: {final_size:.2f} MB")


def main():
    input_files = [
        'Image.jpg',
        'Image (2).jpg',
        'Image (3).jpg',
        'Image (4).jpg',
        'Image (5).jpg',
        'Image (6).jpg',
        'Image (7).jpg',
        'Image (8).jpg',
        'Image (9).jpg',
        'Image (10).jpg',
        'Image (11).jpg',
        'Image (12).jpg',
        'Image (13).jpg',
    ]

    output_file = 'output.tiff'
    target_size = 5.55  # MB

    processor = ImageProcessor(input_files, output_file, target_size)
    processor.process()


if __name__ == "__main__":
    main()


#Fourth

In [ ]:
from PIL import Image
import os


def convert_to_multipage_tiff(input_files, output_file, target_size_mb):
    images = []
    for file in input_files:
        img = Image.open(file)
        images.append(img)

    # First, save without compression
    images[0].save(output_file, save_all=True, append_images=images[1:], compression=None)

    # Check file size
    current_size = os.path.getsize(output_file) / (1024 * 1024)  # Size in MB

    if current_size > target_size_mb:
        # If file is too large, we need to compress slightly
        compression_quality = 100
        while current_size > target_size_mb and compression_quality > 0:
            images[0].save(output_file, save_all=True, append_images=images[1:],
                           compression="jpeg", quality=compression_quality)
            current_size = os.path.getsize(output_file) / (1024 * 1024)
            compression_quality -= 1

    print(f"Final file size: {current_size:.2f} MB")


input_files = [f"Image{i + 1}.jpg" for i in range(13)]

# Target size in MB
target_size = 5.55

output_file = "output.tiff"

convert_to_multipage_tiff(input_files, output_file, target_size)

#Fifth

In [ ]:
from PIL import Image
import os
from google.colab import files


# Interface for Image Compression
class ImageCompressor:
    def compress_image(self, image, compression_type):
        raise NotImplementedError("Subclasses should implement this!")


# TIFF Converter for Zlib (DEFLATE) and LZMA
class JpegToTiffConverter(ImageCompressor):
    def compress_image(self, image, compression_type, quality=95):
        image = image.convert("RGB")
        # Apply compression based on the type selected
        if compression_type == "lzma":
            return self._compress_with_lzma(image)
        else:
            return image.save(compression_type, compression=compression_type, quality=quality)

    def _compress_with_lzma(self, image):
        # Save the image to a buffer with LZMA compression
        import lzma
        import io
        buffer = io.BytesIO()
        image.save(buffer, format="TIFF", compression="tiff_deflate")
        compressed_data = lzma.compress(buffer.getvalue(), preset=9)  # Maximum compression
        return compressed_data

    def convert_images_to_tiff(self, image_list, output_file, compression="tiff_deflate", quality=95):
        # Save images as multi-page TIFF with DEFLATE or LZMA compression
        if compression == "lzma":
            compressed_images = []
            for img in image_list:
                compressed_images.append(self._compress_with_lzma(img))
            # Save compressed LZMA images to TIFF
            with open(output_file, 'wb') as f:
                for img in compressed_images:
                    f.write(img)
        else:
            if image_list:
                image_list[0].save(output_file, save_all=True, append_images=image_list[1:], compression=compression,
                                   quality=quality)
                print(f"Images successfully saved to {output_file}")


# Interface Segregation for different operations
class ImageLoader:
    def load_images(self, uploaded_files):
        raise NotImplementedError("Subclasses should implement this!")


class JpegImageLoader(ImageLoader):
    def load_images(self, uploaded_files):
        image_list = []
        for filename, file_data in uploaded_files.items():
            with open(filename, 'wb') as f:
                f.write(file_data)  # Save the file temporarily
            with Image.open(filename) as img:
                # No resizing, just converting to RGB
                img = img.convert("RGB")
                image_list.append(img)
        return image_list


# Entry point with Dependency Injection
class ImageProcessor:
    def __init__(self, converter: ImageCompressor, loader: ImageLoader):
        self.converter = converter
        self.loader = loader

    def process_images(self, uploaded_files, output_file, compression, quality):
        image_list = self.loader.load_images(uploaded_files)
        self.converter.convert_images_to_tiff(image_list, output_file, compression, quality)


def main():
    print("Please upload your image files (JPG, PNG):")
    uploaded_files = files.upload()

    output_file = "output_compressed_lzma.tif"

    # "zlib" (for DEFLATE) or "lzma" for LZMA compression
    compression = "lzma"

    # Dependency Injection for flexibility
    converter = JpegToTiffConverter()
    loader = JpegImageLoader()
    processor = ImageProcessor(converter, loader)

    processor.process_images(uploaded_files, output_file, compression, quality=95)
    print(f"Conversion completed. Output saved as {output_file}.")


if __name__ == "__main__":
    main()


#imagemagick

In [ ]:
!apt-get update
!apt-get install -y imagemagick

In [ ]:
!convert --version

In [ ]:
# Step 4: Use ImageMagick to create a multi-page TIFF
import subprocess


def compress_images(input_files, output_file, quality=85):
    command = ["convert"] + input_files + ["-compress", "jpeg", "-quality", str(quality), output_file]
    subprocess.run(command)


input_files = [
    'Image.jpg',
    'Image (2).jpg',
    'Image (3).jpg',
    'Image (4).jpg',
    'Image (5).jpg',
    'Image (6).jpg',
    'Image (7).jpg',
    'Image (8).jpg',
    'Image (9).jpg',
    'Image (10).jpg',
    'Image (11).jpg',
    'Image (12).jpg',
    'Image (13).jpg',
]

output_file = 'output.tiff'
compress_images(input_files, output_file)
print(f"Created {output_file} successfully.")

In [ ]:
!convert Image.jpg Image2.jpg Image3.jpg Image4.jpg Image5.jpg Image6.jpg Image7.jpg Image8.jpg Image9.jpg Image10.jpg Image11.jpg Image12.jpg Image13.jpg -compress jpeg -quality 85 output.tiff



#Edited First Method

In [ ]:
from PIL import Image
import os

# Set Default DPI
dpi_x = 100
dpi_y = 100


# Interface for Image Compression
class ImageCompressor:
    def compress_image(self, image, compression_level):
        raise NotImplementedError("Subclasses should implement this!")


# JPEG/PNG to TIFF Converter implementing the Image Compressor Interface
class JpegToTiffConverter(ImageCompressor):
    global dpi_y
    global dpi_x

    def compress_image(self, image, compression_level):
        return image.convert("YCbCr").save(compression_level)

    def convert_images_to_tiff(self, image_list, output_file, compression="jpeg", quality=70):
        # Save images as multi-page TIFF with compression
        if image_list:
            image_list[0].save(output_file, save_all=True, append_images=image_list[1:],
                               dpi=(dpi_x, dpi_y),
                               compression=compression, quality=quality)
            print(f"Images successfully saved to {output_file}")


# Interface Segregation for different operations
class ImageLoader:
    def load_images(self, folder_path):
        raise NotImplementedError("Subclasses should implement this!")


class JpegImageLoader(ImageLoader):
    def load_images(self, folder_path):
        global dpi_y
        global dpi_x

        image_list = []
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(folder_path, filename)
                with Image.open(file_path) as img:
                    # Resize the image to reduce file size
                    # img = img.resize((int(img.width * 0.70), int(img.height * 0.70)))  # Resize to 75% of the original
                    try:
                        dpi_info = img.info['dpi']

                        dpi_x = round(dpi_info[0])
                        dpi_y = round(dpi_info[1])
                    except:
                        pass

                    compressed_img = img.convert("YCbCr")
                    image_list.append(compressed_img)
        return image_list


# Entry point with Dependency Injection
class ImageProcessor:
    def __init__(self, converter: ImageCompressor, loader: ImageLoader):
        self.converter = converter
        self.loader = loader

    def process_images(self, folder_path, output_file, compression, quality):
        image_list = self.loader.load_images(folder_path)
        self.converter.convert_images_to_tiff(image_list, output_file, compression, quality)


def main():
    folder_path = "Images"

    output_file = "output_compressed.tif"

    compression = "jpeg"  # Using lossy JPEG compression within the TIFF
    quality = 30  # Adjust quality; lower values ​​= higher compression

    # Dependency Injection for flexibility
    converter = JpegToTiffConverter()
    loader = JpegImageLoader()
    processor = ImageProcessor(converter, loader)

    processor.process_images(folder_path, output_file, compression, quality)
    print(f"Conversion completed. Output saved as {output_file}.")


if __name__ == "__main__":
    main()
